Validation/examples showing useful identities as part of GWAS regression optimization techniques.

References:
    
- [REGENIE](https://www.biorxiv.org/content/10.1101/2020.06.19.162354v2.full.pdf) - see "Whole genome linear regression" in methods
- [BOLT-LMM](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4342297/#SD1) - see "Standard mixed model association methods"
- [GWAS on your notebook: fast semi-parallel linear and logistic regression for genome-wide association studies](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3695771/)
    - Clear reference on solving multiple regressions simultaneously in R
- [EOSL](https://web.stanford.edu/~hastie/ElemStatLearn//printings/ESLII_print10.pdf) - Section 3.2 "Linear Methods for Regression"

In [2]:
!pip install -q statsmodels

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
from scipy import stats
from dask.array import stats as dask_stats
import statsmodels.api as sm

### Covariate Projection

This shows how for a standard OLS problem, any number of covariates can be removed by projecting the other covariates and outcome.

In [291]:
np.random.seed(0)
n, m = 100, 10
x = np.random.normal(size=(n, m))
b = np.random.normal(size=m)
e = np.random.normal(size=n, scale=1)
y = x @ b + e
x.shape, y.shape

((100, 10), (100,))

In [292]:
b

array([ 0.556,  0.892, -0.422,  0.105,  0.228,  0.201,  0.541, -1.818,
       -0.049,  0.239])

In [293]:
def solve(x, y):
    return np.linalg.lstsq(x, y, rcond=None)[0].squeeze()
np.stack([b, solve(x, y)], axis=1)

array([[ 0.556,  0.604],
       [ 0.892,  0.83 ],
       [-0.422, -0.449],
       [ 0.105,  0.125],
       [ 0.228,  0.286],
       [ 0.201,  0.269],
       [ 0.541,  0.532],
       [-1.818, -1.84 ],
       [-0.049, -0.145],
       [ 0.239,  0.284]])

In [294]:
def project(xs, x, y):
    # Compute (n, n) projection matrix as I - X(XtX)^-1Xt
    p = np.eye(xs.shape[0]) - xs @ np.linalg.inv(xs.T @ xs) @ xs.T
    print(p.shape)
    return p @ x, p @ y

#xp, yp = project(x[:, :-1], x[:, -1], y)
xp, yp = project(x[:, :-2], x[:, -2:], y)
xp.shape, yp.shape

(100, 100)


((100, 2), (100,))

In [295]:
#solve(np.expand_dims(xp, -1), yp)
solve(xp, yp)

array([-0.145,  0.284])

### T-Stats and P-values

This will show how to compute stats/p-values as compared to statsmodels

In [296]:
def regression(x, y):
    n = y.shape[0]
    k = x.shape[1]
    b = solve(x, y)
    yr = y - x @ b
    bv = np.diag(np.linalg.inv(x.T @ x))
    # Make this n - k to equal statsmodels
    sigma2 = np.sum(yr**2) / (n - k - 1)
    t = b / np.sqrt(sigma2 * bv)
    p = 2 * stats.t.sf(np.abs(t), n - k - 1)
    return t, p
tstat, pval = regression(x, y)
tstat, pval

(array([  6.303,   7.951,  -4.14 ,   1.292,   2.797,   2.739,   4.567,
        -19.747,  -1.566,   2.616]),
 array([1.093e-08, 5.463e-12, 7.876e-05, 1.996e-01, 6.326e-03, 7.452e-03,
        1.583e-05, 2.782e-34, 1.209e-01, 1.045e-02]))

In [297]:
with np.printoptions(precision=4, suppress=True):
    print(pval)

[0.     0.     0.0001 0.1996 0.0063 0.0075 0.     0.     0.1209 0.0105]


In [299]:
model = sm.OLS(y, x, hasconst=False)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.871
Model:                            OLS   Adj. R-squared (uncentered):              0.856
Method:                 Least Squares   F-statistic:                              60.68
Date:                Fri, 03 Jul 2020   Prob (F-statistic):                    1.24e-35
Time:                        20:32:00   Log-Likelihood:                         -132.18
No. Observations:                 100   AIC:                                      284.4
Df Residuals:                      90   BIC:                                      310.4
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [300]:
results.pvalues

array([9.043e-09, 4.155e-12, 7.176e-05, 1.971e-01, 6.036e-03, 7.124e-03,
       1.418e-05, 1.193e-34, 1.189e-01, 1.002e-02])

### Simultaneous Regressions

In [5]:
def get_test_data():
    np.random.seed(0)    
    # samples, variants, covs
    n, m, p = 100, 10, 3
    g = np.random.normal(size=(n, m))
    x = np.random.normal(size=(n, p-1))
    # Add intercept (crucial for projection residuals to have mean 0)
    x = np.concatenate([x, np.ones((n, 1))], axis=1)
    bg = np.random.normal(size=m)
    bg[1:4] = 0
    bx = np.random.normal(size=p)
    e = np.random.normal(size=n, scale=.01)

    # Simulate y values using each variant independently
    ys = []
    for i in range(m):
        ys.append(g[:,i] * bg[i] + x @ bx + e)
    return n, m, g, x, bg, bx, ys

n, m, g, x, bg, bx, ys = get_test_data()
g.shape, x.shape, ys[0].shape

((100, 10), (100, 3), (100,))

In [6]:
# Show that OLS for predicting g from x is the same 
# as projection formula
bxg1 = np.linalg.lstsq(x, g, rcond=None)[0] # Regressions are independent if g.shape[1] > 1
bxg2 = (np.linalg.inv(x.T @ x) @ x.T @ g)
np.allclose(bxg1, bxg2)

True

In [9]:
def linear_regression(X, Z, y):
    #####
    # Define projection matrix used to remove common covariates 
    # from individual regressions    
    #####
    
    n = y.shape[0]
    nX = X.shape[1]
    nZ = Z.shape[1]
    
    # Method 1
    # When multiplied by a matrix, this gives the OLS residuals
    # from trying to predict that matrix with Z
    # P = np.eye(Z.shape[0]) - Z @ np.linalg.inv(Z.T @ Z) @ Z.T # has shape (n, n)
    # Xp, yp = P @ X, P @ y
    
    # Method 2
    # Use QR decomp to avoid needing n_samples x n_sample result
    Xp = X - Z @ da.linalg.lstsq(da.array(Z), da.array(X))[0].compute() 
    yp = y - Z @ da.linalg.lstsq(da.array(Z), da.array(y))[0].compute()
    
    # This must be true for the denominator in beta
    # calculation to not require mean terms
    # (it won't be true if X does not contain intercept)
    assert np.allclose(Xp.mean(axis=0), 0)
    assert np.allclose(yp.mean(), 0)
    
    #####
    # Estimate betas
    #####
    Xps = np.sum(Xp**2, axis=0)
    b = (Xp.T @ yp) / Xps
    
    #####
    # P values
    #####
    
    # Method 1
    # Separate calculations per loop covariate
    #     tv, pv = [], []
    #     for i in range(nX):
    #         rss = np.sum((yp - Xp[:,i] * b[i])**2)
    #         sigma2 = rss / (n - nZ - 2)  # This is -2 due to extra -1 from inclusion of variant
    #         t = b[i] / np.sqrt(sigma2 / Xps[i])
    #         p = 2 * stats.t.sf(np.abs(t), n - nZ - 1)
    #         tv.append(t)
    #         pv.append(p)
    #     tv, pv = np.array(tv), np.array(pv)
    
    # Method 2
    # Vectorized calculations
    # Compute sum((y - prediction)**2) from each variant seperately
    rss = np.sum((np.expand_dims(yp, -1) - Xp * b)**2, axis=0)
    sigma2 = rss / (n - nZ - 1)
    tv = b / (np.sqrt(sigma2 / Xps))
    #pv = 2 * stats.t.sf(np.abs(tv), n - nZ - 1)
    pv = 2 * dask_stats.distributions.t.sf(da.absolute(da.array(tv)), n - nZ - 1)#.compute()
    
    return b, tv, pv

df = []
for i in range(m):
    bg_all, t_all, p_all = linear_regression(g, x, ys[i])
    est = sm.OLS(ys[i], np.concatenate([g[:, [i]], x], axis=1), hasconst=True)
    res = est.fit()
    df.append(dict(
        b_true=bg[i],
        b_pred=bg_all[i],
        t_true=res.tvalues[0],
        t_pred=t_all[i],
        p_true=res.pvalues[0],
        p_pred=p_all[i],
    ))
df = pd.DataFrame(df)
df

,b_true,b_pred,t_true,t_pred,p_true,p_pred
0,-0.896370,-0.896579,-884.244646,-884.244646,1.531812e-189,1.531812e-189
1,0.000000,-0.000581,-0.552341,-0.552341,5.819983e-01,5.819983e-01
2,0.000000,0.000131,0.117069,0.117069,9.070498e-01,9.070498e-01
3,0.000000,0.000244,0.249234,0.249234,8.037122e-01,8.037122e-01
4,-1.139008,-1.138473,-1107.434894,-1107.434894,6.346439e-199,6.346439e-199
5,-1.214401,-1.214564,-1198.960640,-1198.960640,3.104859e-202,3.104859e-202
6,0.870962,0.870787,731.088234,731.088234,1.299831e-181,1.299831e-181
7,-0.877971,-0.877598,-909.059750,-909.059750,1.075035e-190,1.075035e-190
8,1.296150,1.296004,1376.889558,1376.889558,5.288514e-208,5.288514e-208
9,0.616459,0.616277,541.865229,541.865229,3.966289e-169,3.966289e-169


In [10]:
assert np.allclose(df['b_true'], df['b_pred'], atol=1e-3)
assert np.allclose(df['t_true'], df['t_pred'])
assert np.allclose(df['p_true'], df['p_pred'])

### Testing

In [45]:
from sgkit.stats import association
import imp
imp.reload(association)

<module 'sgkit.stats.association' from '/home/jovyan/work/repos/sgkit/sgkit/stats/association.py'>

In [34]:
def generate_test_dataset():
    n, m, g, x, bg, bx, ys = get_test_data()
    data_vars = {}
    data_vars['dosage'] = (['variant', 'sample'], g.T)
    for i in range(x.shape[1]):
        data_vars[f'covar_{i}'] = (['sample'], x[:, i])
    for i in range(len(ys)):
        data_vars[f'trait_{i}'] = (['sample'], ys[i])
    return xr.Dataset(data_vars)

ds = generate_test_dataset()
ds

<xarray.Dataset>
Dimensions:  (sample: 100, variant: 10)
Dimensions without coordinates: sample, variant
Data variables:
    dosage   (variant, sample) float64 1.764 0.144 -2.553 ... -0.3031 -0.3581
    covar_0  (sample) float64 0.556 -0.4223 0.2281 ... 1.393 0.01879 -2.012
    covar_1  (sample) float64 0.8925 0.1047 0.2015 ... 1.038 -0.5938 0.5897
    covar_2  (sample) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    trait_0  (sample) float64 -0.7215 -0.1264 2.685 ... 1.954 -0.367 -1.047
    trait_1  (sample) float64 0.8598 0.002675 0.3962 ... 1.365 -0.04471 -0.652
    trait_2  (sample) float64 0.8598 0.002675 0.3962 ... 1.365 -0.04471 -0.652
    trait_3  (sample) float64 0.8598 0.002675 0.3962 ... 1.365 -0.04471 -0.652
    trait_4  (sample) float64 -1.267 -0.5029 -2.189 ... -0.9294 1.024 -0.9813
    trait_5  (sample) float64 2.047 -0.4025 2.162 ... 1.737 -0.5707 -1.153
    trait_6  (sample) float64 1.687 1.304 0.4361 ... 1.011 -0.3983 -0.8248
    trait_7  (sample) float64 0.9927 0.1828 0.5606 ... 2.123 -0.6807 -0.7347
    trait_8  (sample) float64 0.726 0.4085 2.383 -0.7487 ... 1.179 1.751 -2.14
    trait_9  (sample) float64 1.113 -0.5238 1.302 ... 1.129 -0.2316 -0.8728

In [44]:
dsr = association.linear_regression(
    ds, 
    covariates=['covar_0'],
    dosage='dosage',
    trait='trait_1',
    add_intercept=False
)
dsr

<xarray.Dataset>
Dimensions:   (sample: 100, variant: 10, variants: 10)
Dimensions without coordinates: sample, variant, variants
Data variables:
    dosage    (variant, sample) float64 1.764 0.144 -2.553 ... -0.3031 -0.3581
    covar_0   (sample) float64 0.556 -0.4223 0.2281 ... 1.393 0.01879 -2.012
    covar_1   (sample) float64 0.8925 0.1047 0.2015 ... 1.038 -0.5938 0.5897
    covar_2   (sample) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    trait_0   (sample) float64 -0.7215 -0.1264 2.685 ... 1.954 -0.367 -1.047
    trait_1   (sample) float64 0.8598 0.002675 0.3962 ... 1.365 -0.04471 -0.652
    trait_2   (sample) float64 0.8598 0.002675 0.3962 ... 1.365 -0.04471 -0.652
    trait_3   (sample) float64 0.8598 0.002675 0.3962 ... 1.365 -0.04471 -0.652
    trait_4   (sample) float64 -1.267 -0.5029 -2.189 ... -0.9294 1.024 -0.9813
    trait_5   (sample) float64 2.047 -0.4025 2.162 ... 1.737 -0.5707 -1.153
    trait_6   (sample) float64 1.687 1.304 0.4361 ... 1.011 -0.3983 -0.8248
    trait_7   (sample) float64 0.9927 0.1828 0.5606 ... 2.123 -0.6807 -0.7347
    trait_8   (sample) float64 0.726 0.4085 2.383 -0.7487 ... 1.179 1.751 -2.14
    trait_9   (sample) float64 1.113 -0.5238 1.302 ... 1.129 -0.2316 -0.8728
    betas     (variants) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    t_values  (variants) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    p_values  (variants) float64 0.9691 0.4857 0.5276 ... 0.773 0.3415 0.946

In [28]:
dsr[['betas', 't_values', 'p_values']].to_dataframe()

,betas,t_values,p_values
variants,,,
0,-0.000209,-0.206432,0.836890
1,-0.000581,-0.552341,0.581998
2,0.000131,0.117069,0.907050
3,0.000244,0.249234,0.803712
4,0.000535,0.520730,0.603754
5,-0.000163,-0.160802,0.872588
6,-0.000175,-0.146545,0.883799
7,0.000373,0.386320,0.700115
8,-0.000146,-0.154883,0.877239


In [55]:
ds = xr.Dataset(data_vars=dict(x=('dim', np.array([1,2,3])), y=('dim', np.array([1,2,3]))))
ds

<xarray.Dataset>
Dimensions:  (dim: 3)
Dimensions without coordinates: dim
Data variables:
    x        (dim) int64 1 2 3
    y        (dim) int64 1 2 3

In [70]:
rs = np.random.RandomState(1)

In [73]:
x = da.asarray(np.array(['z', 'x', 'x', 'y']))
da.unique(x, return_inverse=True)[1].compute()

array([2, 0, 0, 1])

In [83]:
np.array(['xx', 'y'], dtype='str')

array(['xx', 'y'], dtype='<U2')

In [84]:
import dask.dataframe as dd

In [ ]:
dd.Series.to_dask_array()